In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with qodo Models

{TODO: Update the links below}

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/Qodo_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fnotebook_template.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/Qodo_intro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/Qodo_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook demonstrates how to deploy and use Qodo's state-of-the-art code embedding models on Google Cloud's Vertex AI platform. You'll learn how to set up, deploy, and make predictions with these specialized embedding models that enhance code retrieval and search capabilities.
In this notebook, you will:

Set up your Google Cloud environment and initialize the Vertex AI SDK
Upload a Qodo model to your Vertex AI Model Registry
Create a Vertex AI endpoint for model deployment
Deploy the Qodo model to your endpoint with appropriate compute resources
Make predictions using the deployed model.

### `Qodo` on Vertex AI

You can deploy the `Qodo` models in your own endpoint.



### Available `Qodo` models

#### `Qodo-Embed-1-7B`
Qodo-Embed-1-7B is a state-of-the-art code embedding model for efficient code & text retrieval, enhancing the search accuracy of RAG methods.



## Objective

This notebook shows how to use **Vertex AI API** to deploy the `qodo` models.

For more information, see the [qodo website](https://www.qodo.ai/blog/qodo-embed-1-code-embedding-code-retreival/).


## Get Started


### Install Vertex AI SDK for Python or other required packages


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

In [ ]:
! pip3 install -U -q httpx

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

In [ ]:
PUBLISHER_NAME = "qodo"  # @param {type:"string"}
PUBLISHER_MODEL_NAME = "Qodo-Embed-1-7B"  # @param ["publisher-model-name-1", "publisher-model-name-2", "test-marketplace-publisher-model-e2e-01"]

available_regions = ["us-central1", "europe-west4"]


### Select a location and a version from the dropdown

In [ ]:
import ipywidgets as widgets
from IPython.display import display

dropdown_loc = widgets.Dropdown(
    options=available_regions,
    description="Select a location:",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_loc_eventhandler(change):
    global LOCATION
    if change["type"] == "change" and change["name"] == "value":
        LOCATION = change.new
        print("Selected:", change.new)

LOCATION = dropdown_loc.value
dropdown_loc.observe(dropdown_loc_eventhandler, names="value")
display(dropdown_loc)

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com"

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")

### Import required libraries

In [ ]:
import base64
import json
import time

import httpx
import requests

## Using Vertex AI API

### Upload Model

In [ ]:
UPLOAD_MODEL_PAYLOAD = {
  "model": {
    "displayName": "ModelGarden_LaunchPad_Model_" + time.strftime("%Y%m%d-%H%M%S"),
    "baseModelSource": {
        "modelGardenSource": {
          "publicModelName": f"publishers/{PUBLISHER_NAME}/models/{PUBLISHER_MODEL_NAME}",
        }
    }
  }
}

request = json.dumps(UPLOAD_MODEL_PAYLOAD)

! curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/models:upload -d '{request}'

### Get Model

In [ ]:
MODEL_ID = 123456789		  # @param {type: "number"}

! curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/models/{MODEL_ID}

### Create Endpoint

In [ ]:
CREATE_ENDPOINT_PAYLOAD = {
  "displayName": "ModelGarden_LaunchPad_Endpoint_" + time.strftime("%Y%m%d-%H%M%S"),
}

request = json.dumps(CREATE_ENDPOINT_PAYLOAD)

! curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints -d '{request}'

### Get Endpoint

In [ ]:
ENDPOINT_ID = 123456789 # @param {type: "number"}

! curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}

### Deploy Model

In [ ]:
MACHINE_TYPE = "a2-highgpu-1g"  # @param {type: "string"}
ACCELERATOR_TYPE = "NVIDIA_Tesla_A100" # @param {type: "string"}
ACCELERATOR_COUNT = 1 # @param {type: "number"}


In [ ]:
DEPLOY_PAYLOAD = {
  "deployedModel": {
    "model": f"projects/{PROJECT_ID}/locations/{LOCATION}/models/{MODEL_ID}",
    "displayName": "ModelGarden_LaunchPad_DeployedModel_" + time.strftime("%Y%m%d-%H%M%S"),
    "dedicatedResources": {
     "machineSpec": {
        "machineType": MACHINE_TYPE,
        "acceleratorType": ACCELERATOR_TYPE,
        "acceleratorCount": ACCELERATOR_COUNT,
     },
     "minReplicaCount": 1,
     "maxReplicaCount": 1
    },
  },
  "trafficSplit": {
    "0": 100
  }
}

request = json.dumps(DEPLOY_PAYLOAD)
print("Request payload to Deploy Model:")
print(json.dumps(DEPLOY_PAYLOAD, indent=2))
print("\nResult:")
! curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:deployModel -d '{request}'

### Prediction

#### Unary call

Sends a POST request to the specified API endpoint to get a response from the model for a joke using the provided payload.

In [ ]:
PAYLOAD = {
   "instances":[
      {
         "inputs":[
            "def hello_world(): \n  print('hello_world')"
         ]
      }
   ]
}


request = json.dumps(PAYLOAD)

!curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:rawPredict \
  -d '{request}'

## Using Vertex AI SDK for *Python*

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)

### Upload Model

In [ ]:
model = aiplatform.Model.upload(
    display_name="ModelGarden_LaunchPad_Endpoint_" + time.strftime("%Y%m%d-%H%M%S"),
    model_garden_source_model_name = f"publishers/{PUBLISHER_NAME}/models/{PUBLISHER_MODEL_NAME}",
)

### Create Endpoint

In [ ]:
my_endpoint = aiplatform.Endpoint.create(display_name="ModelGarden_LaunchPad_Endpoint_" + time.strftime("%Y%m%d-%H%M%S"))

### Deploy Model

In [ ]:
MACHINE_TYPE = "a2-highgpu-1g"  # @param {type: "string"}
ACCELERATOR_TYPE = "NVIDIA_Tesla_A100" # @param {type: "string"}
ACCELERATOR_COUNT = 1 # @param {type: "number"}

In [ ]:
model.deploy(
    endpoint=my_endpoint,
    deployed_model_display_name="ModelGarden_LaunchPad_DeployedModel_" + time.strftime("%Y%m%d-%H%M%S"),
    traffic_split={"0": 100},
    machine_type=MACHINE_TYPE,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=ACCELERATOR_COUNT,
    min_replica_count=1,
    max_replica_count=1,
)

### Prediction

#### Unary call

In [ ]:
PAYLOAD = {
   "instances":[
      {
         "inputs":[
            "def hello_world(): \n  print('hello_world')"
         ]
      }
   ]
}


request = json.dumps(PAYLOAD)

response = my_endpoint.raw_predict(
    body = request,
    headers = {'Content-Type':'application/json'}
)
embedding = response['predictions'][0]['data'][0]['embedding']
print(embedding)